# The Characteristics of a Tennis Match Winner

### Group 38 Project Proposal

#### Introduction
Likelihood of winning a game of tennis can be divided into four categories: strategy, technique, physical attributes and preparation, and mental game. While all components contribute to a win, we aim to test the importance of physical attributes and preparation by predicting the current rank of a tennis player based on attributes such as height, age, hand-dominance, seasons active, and favorite surface. To answer this question we will use data from the dataset “player_stats.csv”, which describes the player statistics for the top 500 tennis players, using data on physical attributes such as height, age, and hand dominance, as well as on variables such as seasons active and favorite surface. 

#### Preliminary exploratory data analysis

##### Methods
In progress

##### Expected outcomes and significance:

- What do you expect to find?
- What impact could such findings have?
- What future questions could this lead to?

#### Our Data Analysis:

In [2]:
# Setup

library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
source("tests.R")
source('cleanup.R')

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


##### First, we will tidy up our data:

In [16]:
# Load the data from the web
url <- "https://drive.google.com/uc?export=download&id=1_MECmUXZuuILYeEOfonSGqodW6qVdhsS"
tennis <- read_csv(url) 

# Fix column names
colnames(tennis) <- make.names(colnames(tennis))

# Select only relevant columns to our study
tennis <- select(tennis, Age:Seasons)
tennis <- select(tennis, c(Age, Plays, Current.Rank, Height))

# Omit any rows with NA (there are still 113 rows left)
tennis <- na.omit(tennis) 

tennis

New names:
• `` -> `...1`
Rows: 500 Columns: 38
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (25): Age, Country, Plays, Wikipedia, Current Rank, Best Rank, Name, Bac...
dbl (13): ...1, Turned Pro, Seasons, Titles, Best Season, Retired, Masters, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Age,Plays,Current.Rank,Height
<chr>,<chr>,<chr>,<chr>
32 (03-11-1987),Right-handed,178 (280),185 cm
27 (21-10-1992),Right-handed,183 (273),193 cm
31 (23-09-1988),Right-handed,121 (460),198 cm
⋮,⋮,⋮,⋮
29 (27-12-1990),Right-handed,35 (1305),196 cm
34 (17-06-1985),Right-handed,179 (279),183 cm
26 (03-09-1993),Right-handed,5 (5890),185 cm


In [13]:
# Now we will tidy up our data further by getting rid of unnecessary information in certain columns (ex. dates, "cm", extra number next to rank)

# Separate unnecessary information from their original column into a new column
tennis <- separate(tennis, col = Age, into = c("Age", "x"), sep = " ", convert = TRUE) |> 
          separate(col = Current.Rank, into = c("Current.Rank", "y"), sep = " ", convert = TRUE) |>
          separate(col = Height, into = c("Height", "z"), sep = " ", convert = TRUE) 

# Delete unnecessary columns
tennis <- select(tennis, -c(x, y, z))

tennis

Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 115 rows [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...].”
Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 115 rows [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...].”
Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 115 rows [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...].”


Age,Plays,Current.Rank,Height
<int>,<chr>,<int>,<int>
32,Right-handed,178,185
27,Right-handed,183,193
31,Right-handed,121,198
⋮,⋮,⋮,⋮
29,Right-handed,35,196
34,Right-handed,179,183
26,Right-handed,5,185
